Загружены данные из AppMetrica за период с 1 января по 31 марта 2020, только по пользователям, зарегистрированным позднее 1 января 2020.

Описание данных 

1. date – дата совершения события

2. event - событие:

    - app_install – установка приложения
    - app_start – открыть приложения
    - registration – зарегистрироваться 
    - search – перейти на страницу поиска товаров (каталог)
    - open_item – открыть товар
    - choose_item – отправить товар в корзину
    - tap_basket – перейти в корзину
3. purchase – подтверждение покупки
4. gender – пол пользователя
5. os_name – платформа пользователя
6. city – город пользователя
7. device_id – идентификатор устройства пользователя
8. urm_source – канал, с которого пришел пользователь

    - yandex-direct – Яндекс директ
    - google_ads – реклама в Google
    - vk_ads – реклама в ВК
    - instagram_ads – реклама в instagram
    - facebook_ads – реклама в facebook
    - referral – акция «приведи друга»
Если стоит ‘-’, то канал не определен или это скачивание приложения напрямую или посещение не с рекламы  
9. purchase_sum – стоимость покупки (при совершении события ‘purchase’)

Обратите внимание на следующее:

- В выгрузке только уникальные действия пользователей за каждый день 

- Можно миновать стадию установки приложения, если оно было установлено ранее

- Можно миновать стадию регистрации, если пользователь был уже залогинен на момент сессии. Однако незарегистрированные пользователи не могут оформить покупку. 

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from datetime import timedelta
import datetime as dt
%matplotlib inline 
import calendar

In [3]:
import requests
from urllib.parse import urlencode

In [4]:
# таблица с уникальными идентификаторами пользователей
df = pd.read_csv('KC_case_data.zip')
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads


In [12]:

df.shape

(2747968, 8)

In [13]:
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads


In [14]:
df.dtypes

date             object
event            object
purchase_sum    float64
os_name          object
device_id         int64
gender           object
city             object
utm_source       object
dtype: object

In [15]:
df['date'] = pd.to_datetime(df['date'])

# 1.Найти MAU февраля

### Note: Месячная аудитория (MAU) — количество уникальных пользователей, которые зашли в приложение в течение месяца. По отношению средней дневной аудитории к месячной можно понимать частоту использования продукта.

In [16]:
df["year_month"] = df["date"].dt.to_period("M")

In [17]:
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,year_month
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,2020-01
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,2020-01
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,2020-01


In [18]:
feb = df[(df.year_month == '2020-02')]\
    .groupby('event')\
    .device_id\
    .nunique()\
    .to_frame()\
    .sort_values('device_id', ascending=False)
feb

,device_id
event,
app_start,75032
search,72666
choose_item,61813
tap_basket,50352
app_install,38078
purchase,27902
register,20578


In [19]:
MAU = feb.query('event == "app_start"')
MAU

,device_id
event,
app_start,75032


# 2.Определить количество установок в январе

In [20]:
jan = df[(df.year_month == '2020-01')]\
    .groupby('event')\
    .device_id\
    .nunique()\
    .to_frame()\
    .sort_values('device_id', ascending=False)
jan

,device_id
event,
app_start,99161
search,95988
choose_item,81872
app_install,80297
tap_basket,66720
register,42946
purchase,39012


In [21]:
install_jan = jan.query('event == "app_install"')
install_jan

,device_id
event,
app_install,80297


# Задание 3 

### Присвойте пользователям когорты по дню установки приложения и посчитайте для них конверсию из установки в покупку в течение 7 дней. Для какой когорты конверсия была наибольшей? Ответ впишите в формате: дд.мм.гггг

Примечание: считаем пользователя сконвертировавшимся, если с момента установки до совершения первой покупки прошло не более 7 дней.

In [22]:
# Создадим отдельный df с установками

df_installs=df.query('event=="app_install"').rename(columns={'date':'install_date'})
df_installs

,install_date,event,purchase_sum,os_name,device_id,gender,city,utm_source,year_month
2515061,2020-01-01,app_install,NaN,ios,4921563,male,Moscow,vk_ads,2020-01
2515062,2020-01-01,app_install,NaN,ios,1311583,male,Moscow,-,2020-01
2515063,2020-01-01,app_install,NaN,android,23006325,male,Saint-Petersburg,-,2020-01
2515064,2020-01-01,app_install,NaN,android,5645652,male,Moscow,-,2020-01
2515065,2020-01-01,app_install,NaN,ios,8798567,male,Saint-Petersburg,referal,2020-01
...,...,...,...,...,...,...,...,...,...
2669653,2020-03-31,app_install,NaN,ios,19717968,female,Moscow,-,2020-03
2669654,2020-03-31,app_install,NaN,android,32966665,female,Saint-Petersburg,-,2020-03
2669655,2020-03-31,app_install,NaN,ios,6335964,female,Moscow,referal,2020-03
2669656,2020-03-31,app_install,NaN,android,29155826,male,Saint-Petersburg,vk_ads,2020-03


In [23]:
# Создадим отдельный df с покупками

df_purchases=df.query('event=="purchase"').rename(columns={'date':'purchase_date'}).sort_values('purchase_date').drop_duplicates('device_id',keep='first')
df_purchases

,purchase_date,event,purchase_sum,os_name,device_id,gender,city,utm_source,year_month
7691,2020-01-01,purchase,265.5,android,6081155,female,Moscow,-,2020-01
8289,2020-01-01,purchase,292.0,ios,1955306,female,Moscow,yandex-direct,2020-01
8290,2020-01-01,purchase,664.0,ios,20176327,female,Moscow,facebook_ads,2020-01
8291,2020-01-01,purchase,337.0,android,28603960,male,Moscow,-,2020-01
8292,2020-01-01,purchase,502.0,ios,14028316,female,Moscow,referal,2020-01
...,...,...,...,...,...,...,...,...,...
2506842,2020-03-31,purchase,670.0,ios,32122725,male,Saint-Petersburg,-,2020-03
2506845,2020-03-31,purchase,407.0,ios,23188703,female,Moscow,instagram_ads,2020-03
2506848,2020-03-31,purchase,149.5,android,25732558,male,Saint-Petersburg,referal,2020-03
2506852,2020-03-31,purchase,442.5,android,17778508,male,Saint-Petersburg,referal,2020-03


In [24]:
# объединим установки и покупки

installs_purchases=df_installs.merge(df_purchases, how='left', on=['device_id'])
installs_purchases

,install_date,event_x,purchase_sum_x,os_name_x,device_id,gender_x,city_x,utm_source_x,year_month_x,purchase_date,event_y,purchase_sum_y,os_name_y,gender_y,city_y,utm_source_y,year_month_y
0,2020-01-01,app_install,NaN,ios,4921563,male,Moscow,vk_ads,2020-01,2020-01-10,purchase,311.0,ios,male,Moscow,-,2020-01
1,2020-01-01,app_install,NaN,ios,1311583,male,Moscow,-,2020-01,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,2020-01-01,app_install,NaN,android,23006325,male,Saint-Petersburg,-,2020-01,2020-01-01,purchase,479.5,android,male,Saint-Petersburg,-,2020-01
3,2020-01-01,app_install,NaN,android,5645652,male,Moscow,-,2020-01,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,2020-01-01,app_install,NaN,ios,8798567,male,Saint-Petersburg,referal,2020-01,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154592,2020-03-31,app_install,NaN,ios,19717968,female,Moscow,-,2020-03,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT
154593,2020-03-31,app_install,NaN,android,32966665,female,Saint-Petersburg,-,2020-03,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT
154594,2020-03-31,app_install,NaN,ios,6335964,female,Moscow,referal,2020-03,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT
154595,2020-03-31,app_install,NaN,android,29155826,male,Saint-Petersburg,vk_ads,2020-03,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [25]:
# интервал между установкой и покупкой

installs_purchases['date_diff']=installs_purchases.purchase_date-installs_purchases.install_date
installs_purchases['date_diff']=installs_purchases.date_diff.dt.days

In [26]:
installs_purchases

,install_date,event_x,purchase_sum_x,os_name_x,device_id,gender_x,city_x,utm_source_x,year_month_x,purchase_date,event_y,purchase_sum_y,os_name_y,gender_y,city_y,utm_source_y,year_month_y,date_diff
0,2020-01-01,app_install,NaN,ios,4921563,male,Moscow,vk_ads,2020-01,2020-01-10,purchase,311.0,ios,male,Moscow,-,2020-01,9.0
1,2020-01-01,app_install,NaN,ios,1311583,male,Moscow,-,2020-01,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
2,2020-01-01,app_install,NaN,android,23006325,male,Saint-Petersburg,-,2020-01,2020-01-01,purchase,479.5,android,male,Saint-Petersburg,-,2020-01,0.0
3,2020-01-01,app_install,NaN,android,5645652,male,Moscow,-,2020-01,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
4,2020-01-01,app_install,NaN,ios,8798567,male,Saint-Petersburg,referal,2020-01,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154592,2020-03-31,app_install,NaN,ios,19717968,female,Moscow,-,2020-03,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
154593,2020-03-31,app_install,NaN,android,32966665,female,Saint-Petersburg,-,2020-03,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
154594,2020-03-31,app_install,NaN,ios,6335964,female,Moscow,referal,2020-03,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
154595,2020-03-31,app_install,NaN,android,29155826,male,Saint-Petersburg,vk_ads,2020-03,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN


In [27]:
# Отфильтруем сконвертировавшихся в течение 7 дней

installs_purchases.date_diff=installs_purchases.date_diff.where(installs_purchases.date_diff<8)

In [28]:

# Сгруппируем по дням (когорты) и посчитаем количество установок и покупок в каждый из дней

i_p=installs_purchases.groupby('install_date',as_index=False).agg({'event_x':'count','date_diff':'count'})
i_p=i_p.rename(columns={'event_x':'installs','date_diff':'purchases'})

In [29]:
i_p

,install_date,installs,purchases
0,2020-01-01,3579,1408
1,2020-01-02,3144,1186
2,2020-01-03,2402,834
3,2020-01-04,1831,639
4,2020-01-05,1671,587
...,...,...,...
86,2020-03-27,1199,277
87,2020-03-28,1091,209
88,2020-03-29,1117,223
89,2020-03-30,994,204


In [30]:
# конверсия (покупки/установки) и отсортируем по убыванию конверсии

i_p['conversion']=round(i_p.purchases/i_p.installs*100, 2)
i_p.sort_values('conversion',ascending=False)

,install_date,installs,purchases,conversion
0,2020-01-01,3579,1408,39.34
8,2020-01-09,1424,558,39.19
14,2020-01-15,4310,1650,38.28
13,2020-01-14,5173,1973,38.14
1,2020-01-02,3144,1186,37.72
...,...,...,...,...
77,2020-03-18,1171,235,20.07
88,2020-03-29,1117,223,19.96
81,2020-03-22,1261,251,19.90
87,2020-03-28,1091,209,19.16


In [31]:
i_p[i_p.conversion == i_p.conversion.max()].install_date

0   2020-01-01
Name: install_date, dtype: datetime64[ns]

# Задание 4
Укажите значение CR из предыдущего вопроса в % (округлите до 1 цифры после запятой). Формат числа - с точкой. Пример: 66.4.

In [32]:
i_p[i_p.conversion == i_p.conversion.max()]

,install_date,installs,purchases,conversion
0,2020-01-01,3579,1408,39.34


# Задание 5

С какого платного маркетингового канала пришло больше всего новых пользователей?

В ответ впишите один из каналов в таком формате: «Яндекс», «Гугл», «ВК», «Фейсбук», «Инстаграм», «Реферальная программа».

NB! В этом и подобном ему заданиях писать надо только название, без кавычек.

In [33]:
df

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,year_month
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,2020-01
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,2020-01
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,2020-01
...,...,...,...,...,...,...,...,...,...
2747963,2020-03-31,register,NaN,android,2984778,male,Saint-Petersburg,facebook_ads,2020-03
2747964,2020-03-31,register,NaN,ios,27301864,male,Moscow,-,2020-03
2747965,2020-03-31,register,NaN,ios,1294285,female,Saint-Petersburg,google_ads,2020-03
2747966,2020-03-31,register,NaN,android,3010574,female,Saint-Petersburg,google_ads,2020-03


In [34]:
utm = df\
    .groupby('utm_source')\
    .device_id\
    .nunique()\
    .to_frame()\
    .sort_values('device_id', ascending=False)
utm

,device_id
utm_source,
-,52273
yandex-direct,40712
google_ads,38096
vk_ads,34086
instagram_ads,31048
facebook_ads,25959
referal,15926


# Задание 6

Проанализируйте на каком этапе воронки отваливается бОльшая часть клиентов. Посмотрите отдельно сценарии для зарегистрированных и для незарегистрированных пользователей. На каком шаге отваливается больше всего зарегистрированных пользователей?

В ответ впишите один из шагов, конверсия из которого (в следующий шаг) самая низкая, в таком формате:
«Поиск», «Добавление товара», «Переход в корзину», «Регистрация».

Подсказка: 

- Для того, чтобы выделить группу тех, кому нужно и не нужно регистрироваться, добавьте колонку с датой регистрации.
Если дата регистрации < даты совершения события, то пользователь уже зарегистрирован. 

In [35]:
df

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,year_month
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,2020-01
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,2020-01
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,2020-01
...,...,...,...,...,...,...,...,...,...
2747963,2020-03-31,register,NaN,android,2984778,male,Saint-Petersburg,facebook_ads,2020-03
2747964,2020-03-31,register,NaN,ios,27301864,male,Moscow,-,2020-03
2747965,2020-03-31,register,NaN,ios,1294285,female,Saint-Petersburg,google_ads,2020-03
2747966,2020-03-31,register,NaN,android,3010574,female,Saint-Petersburg,google_ads,2020-03


In [36]:
reg_users = df[df.event == 'register'].device_id.unique()

In [37]:
df_reg_users = df[df.device_id.isin(reg_users)] # is device_id in reg_users

In [38]:
df[~df.device_id.isin(reg_users)].event.unique()

array(['app_start', 'choose_item', 'search', 'tap_basket', 'app_install'],
      dtype=object)

In [39]:
df.event.unique()

array(['app_start', 'choose_item', 'purchase', 'search', 'tap_basket',
       'app_install', 'register'], dtype=object)

In [40]:
last_event_indexes = df_reg_users\
.sort_values(['device_id', 'date'])\
.reset_index(drop=True).reset_index()\
.groupby(['device_id']).agg({'index':'max'})['index']

In [41]:
last_event_indexes

device_id
4014            104
4046            232
4060            242
4067            248
4120            290
             ...   
35342310    2205937
35358658    2205943
35367797    2205950
35370030    2205957
35381595    2205965
Name: index, Length: 78310, dtype: int64

In [42]:
t = df_reg_users\
.sort_values(['device_id', 'date'])\
.reset_index(drop=True).reset_index()\
[df_reg_users.index.isin(last_event_indexes)].event.value_counts()

In [43]:
t.to_frame().query("index not in ['app_start', 'app_install']")\
.assign(next=lambda x: x.event.shift(-1), ratio=lambda x: x.next / x.event)

,event,next,ratio
search,16785,13087.0,0.779684
choose_item,13087,9546.0,0.729426
tap_basket,9546,4175.0,0.437356
purchase,4175,2262.0,0.541796
register,2262,NaN,NaN


# Задание 7

Пользователи, пришедшие с каких каналов, показали самую низкую конверсию в первую покупку?

В ответ впишите один из каналов в таком формате: «Яндекс», «Гугл», «ВК», «Фейсбук», «Инстаграм», «Реферальная программа».

Подсказка: 

- CR = кол-во юзеров, впервые совершивших покупку/кол-во юзеров, впервые открывших приложение 

In [44]:
df_pivot = df.groupby(by=['utm_source', 'event'], as_index=False) \
    .agg({'device_id':'nunique'}) \
    .rename(columns={'device_id':'counts'}) \
    .pivot(index='utm_source', columns='event', values='counts')

df_pivot

event,app_install,app_start,choose_item,purchase,register,search,tap_basket
utm_source,,,,,,,
-,32460,52268,41276,16598,16527,50282,32076
facebook_ads,13916,25957,20687,9017,8672,24963,16362
google_ads,26286,38092,29495,11339,11405,36495,22588
instagram_ads,20096,31045,24503,10762,10750,29805,19339
referal,9282,15924,12849,6362,6193,15358,10395
vk_ads,23189,34079,27001,12364,12421,32713,21440
yandex-direct,29368,40707,31501,12028,12342,39018,24110


In [45]:
(df_pivot.purchase / df_pivot.app_start) \
    .to_frame() \
    .reset_index() \
    .rename(columns={0:'cr'}) \
    .sort_values(by='cr')

,utm_source,cr
6,yandex-direct,0.295477
2,google_ads,0.297674
0,-,0.317556
3,instagram_ads,0.346658
1,facebook_ads,0.347382
5,vk_ads,0.362804
4,referal,0.399523


# Задание 8


Пользователи, пришедшие с какого канала, имеют медианный первый чек выше?

(учитываются только первые покупки пользователей)

In [46]:
df_purchase = df.query('event == "purchase"') \
    .groupby(by='device_id', as_index=False) \
    .agg({'date':'min'}) \
    .rename(columns={'date':'purchase_date'})

df = pd.merge(df, df_purchase, on='device_id')

In [47]:
df.query('event == "purchase" and date == purchase_date').groupby(by='utm_source') \
    .agg({'purchase_sum':'median'}) \
    .sort_values(by='purchase_sum')

,purchase_sum
utm_source,
facebook_ads,389.0
google_ads,390.5
yandex-direct,392.5
vk_ads,393.0
instagram_ads,393.5
referal,395.5
-,398.5


# Задание 9

Данные по затратам на рекламу:

- Яндекс – 10 491 707 руб.

- Гугл – 10 534 878 руб.

- Фейсбук – 8 590 498 руб.

- Инстаграм – 8 561626 руб.

- ВК – 9 553 531руб.

Расходы на реферальную программу: если пользователь приведет друга и последний совершит первую покупку, то оба получат по 100 рублей.

Какой платный канал привлечения (среди рекламных) имеет самый высокий ROMI? 

In [48]:
map_cost = {
    'yandex-direct' : 10491707,
    'google_ads' : 10534878,
    'facebook_ads' : 8590498,
    'instagram_ads' : 8561626,
    'vk_ads': 9553531
}

In [49]:
df_pur = df.groupby('utm_source', as_index=False).agg({'purchase_sum':'sum'})

In [50]:
df_pur['cost'] = df_pur.utm_source.map(map_cost)

In [51]:
df_pur.head(3)

,utm_source,purchase_sum,cost
0,-,21449749.5,NaN
1,facebook_ads,12249901.0,8590498.0
2,google_ads,12868276.0,10534878.0


ROMI = (валовая прибыль - маркетинговые расходы)/маркетинговые расходы

- валовая прибыль: df.purchase_sum
- маркетинговые расходы: df_pur.cost

In [52]:
df_pur['romi'] = (df_pur.purchase_sum - df_pur.cost) / df_pur.cost
df_pur.sort_values('romi', ascending=False)

,utm_source,purchase_sum,cost,romi
5,vk_ads,16389652.5,9553531.0,0.715560
3,instagram_ads,14546969.0,8561626.0,0.699090
1,facebook_ads,12249901.0,8590498.0,0.425983
6,yandex-direct,13915368.0,10491707.0,0.326321
2,google_ads,12868276.0,10534878.0,0.221493
0,-,21449749.5,NaN,NaN
4,referal,8837044.5,NaN,NaN


Ответ:


In [53]:
df_pur[df_pur.romi == df_pur.romi.max()]

,utm_source,purchase_sum,cost,romi
5,vk_ads,16389652.5,9553531.0,0.71556
